# Progetto di Foundation of computer science

### Corso di laurea in Data Science A.A 2019/2020

### Simone D'Amico - 850369

In [1]:
import time
start = time.time()

In [2]:
# Import delle libreire usate
import pandas as pd
import numpy as np

In [3]:
# Caricamento del tempo di esecuzione di ogni cella
%load_ext autotime

# Definizione del formato per i numeri decimali, si mostra fino alla terza cifra decimale inclusa
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Caricamento dei dataset utilizzati

In [4]:
# Per motivi di tempo e di memoria sono caricate dai dataset solo le colonne usate per i vari task

df_loans_lenders = pd.read_csv("loans_lenders.csv")

use_col_loans =["loan_id", "loan_name", "disburse_time", "planned_expiration_time", 
                "country_name", "loan_amount", "country_code",]
date_type = ["disburse_time", "planned_expiration_time"]
df_loans = pd.read_csv("loans.csv", usecols=use_col_loans, parse_dates=date_type)


use_col_lenders =["permanent_name", 'city', 'state', "country_code"]
df_lenders = pd.read_csv("lenders.csv", usecols=use_col_lenders)


used_col_country = ['country_name', 'country_code', 'population', 'population_below_poverty_line']
df_country_stats = pd.read_csv('country_stats.csv', usecols=used_col_country)

time: 19.4 s


### 1. Normalize the loan_lenders table. In the normalized table, each row must have one `loan_id` and one lender.

In [5]:
# Partendo dal dataset df_loans_lenders si usano come indici loan_id, così si conservano durante l'esecuzione. 
# Ogni valore di lenders viene trasformato in una lista e, grazie a explode, 
# si ottengono le coppie (loan, specifico lender)
df_loan_lender_norm = df_loans_lenders.set_index(['loan_id']).apply(
    lambda x: x.str.split(", ").explode()).reset_index()

time: 32.2 s


In [6]:
df_loan_lender_norm.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


time: 29.1 ms


### 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [7]:
# Si calcola la differenza in giorni tra le colonne delle date e si considerano i giorni
df_loans["diff_days"] = (df_loans["planned_expiration_time"] - df_loans["disburse_time"]).dt.days

time: 76.5 ms


In [8]:
df_loans.head()

,loan_id,loan_name,loan_amount,country_code,country_name,planned_expiration_time,disburse_time,diff_days
0,657307,Aivy,125.000,PH,Philippines,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,53.000
1,657259,Idalia Marizza,400.000,HN,Honduras,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,96.000
2,658010,Aasia,400.000,PK,Pakistan,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,37.000
3,659347,Gulmira,625.000,KG,Kyrgyzstan,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,34.000
4,656933,Ricky\t,425.000,PH,Philippines,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,57.000


time: 15 ms


In [9]:
# Per verificare che le righe della colonna diff_days siano mancanti se almeno una delle colonne 
# planned_expiration_time o disburse_time è mancante si può usare la funzione info()

df_loans[df_loans['planned_expiration_time'].isnull()].info()

print("\n--------------------------------------------------------------\n")

df_loans[df_loans['disburse_time'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371834 entries, 64 to 1419523
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype              
---  ------                   --------------   -----              
 0   loan_id                  371834 non-null  int64              
 1   loan_name                355759 non-null  object             
 2   loan_amount              371834 non-null  float64            
 3   country_code             371834 non-null  object             
 4   country_name             371834 non-null  object             
 5   planned_expiration_time  0 non-null       datetime64[ns, UTC]
 6   disburse_time            371832 non-null  datetime64[ns, UTC]
 7   diff_days                0 non-null       float64            
dtypes: datetime64[ns, UTC](2), float64(2), int64(1), object(3)
memory usage: 25.5+ MB

--------------------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2813 entries, 31

### 3. Find the lenders that have funded at least twice.

In [10]:
# Si raggruppa per lender e si ottiene il numero di record di ogni gruppo
count = df_loan_lender_norm.groupby(['lenders']).size()

# Si selezionano solo i lenders che hanno finanziato due o più volte 
count = count[count >=2]

# Si definisce un dataset per mostrare il risultato
df_count = count.reset_index(name='count')

time: 10.5 s


### 4. For each country, compute how many loans have involved that country as borrowers.

In [11]:
# Si può usare value_count applicato alla colonna country_name per contare le occorrenze di ogni paese 
# ottienendo un dataframe 
df_count_loans = df_loans["country_name"].value_counts().reset_index()

df_count_loans = df_count_loans.rename(columns= {'index': 'country_name', 'country_name': 'count'})

time: 161 ms


In [12]:
df_count_loans.head()

,country_name,count
0,Philippines,285336
1,Kenya,143699
2,Peru,86000
3,Cambodia,79701
4,El Salvador,64037


time: 7.11 ms


### 5. For each country, compute the overall amount of money borrowed.

In [13]:
# Si raggruppano i loans per country_name e si calcola la somma di loan_amount per ogni gruppo
df_overall_borrowed = df_loans.groupby("country_name")["loan_amount"].sum().reset_index(name='tot_borrow')

time: 110 ms


In [14]:
df_overall_borrowed.head()

,country_name,tot_borrow
0,Afghanistan,1967950.000
1,Albania,4307350.000
2,Armenia,22950475.000
3,Azerbaijan,14784625.000
4,Belize,150175.000


time: 6.78 ms


### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [15]:
# Si calcola la somma totale dei prestiti
tot_borrow = df_overall_borrowed["tot_borrow"].sum()

# Si calcola la percentuale di prestiti di ogni stato rispetto alla somma totale
df_overall_borrowed["perc_borrow"] = (df_overall_borrowed["tot_borrow"]/tot_borrow)*100

time: 4.83 ms


In [16]:
df_overall_borrowed.head()

,country_name,tot_borrow,perc_borrow
0,Afghanistan,1967950.000,0.167
1,Albania,4307350.000,0.365
2,Armenia,22950475.000,1.943
3,Azerbaijan,14784625.000,1.251
4,Belize,150175.000,0.013


time: 8.63 ms


### 7. Like the three previous points, but split for each year (with respect to disburse time).

In [17]:
# Alcune osservazioni sono mancanti per la colonna disburse_time.
# Si vedono le proporzioni di quesi valori mancanti rispetto a tutto il dataset
# sia per il numero di righe che per l'ammontare del prestito a cui non è associato nessuna data.

df_missing_disburse_time = df_loans[df_loans['disburse_time'].isna()]

row_missing = df_missing_disburse_time.shape[0]
loan_amount_missing = df_missing_disburse_time['loan_amount'].sum()

# Osservazioni mancanti rispetto alle righe del dataset
print(f"Righe totali del dataset: {format(df_loans.shape[0], ',d')}")
print(f"Righe mancanti: {format(row_missing, ',d')}")
print(f"Percentuale di righe mancanti: {round(((row_missing/df_loans.shape[0])*100), 3)}%\n")

# Osservazioni mancanti rispetto ai prestiti
print(f"Totale prestiti: {format(df_loans['loan_amount'].sum(), ',.3f')}")
print(f"Prestiti senza anno: {format(loan_amount_missing, ',.3f')}")
print(f"Percentuale prestiti senza anno: {format(round(((loan_amount_missing/df_loans['loan_amount'].sum())*100), 3), '.3f')}%")

Righe totali del dataset: 1,419,607
Righe mancanti: 2,813
Percentuale di righe mancanti: 0.198%

Totale prestiti: 1,181,437,300.000
Prestiti senza anno: 10,940,750.000
Percentuale prestiti senza anno: 0.926%
time: 26.7 ms


In [18]:
# Dato l'esiguo numero di valori mancanti si è scelto di non operare nessuna sostituzione di tali valori
# ma di considerarli invece allo stesso livello dei gruppi degli anni.

# Si effettua il group by sulla data da cui è stato estratto l'anno. Si usa dt per gestire le date nelle 
# pandas Series. Per ogni gruppo si calcolano il numero di righe e la somma dei loan
df_group = df_loans.groupby(df_loans["disburse_time"].dt.year)["loan_amount"].agg([('count', 'count'), 
                                                                            ('loan_amount', 'sum')])

# Cast degli anni ad intero
df_group.index = df_group.index.astype(int)

# Al dataset finale si aggiunge una riga con le statistiche per le osservazioni mancanti.
df_group = df_group.append(pd.Series({'count':row_missing, 
                                      'loan_amount':loan_amount_missing}, name="missing_year"))

# Si calcola la percentuale di loan amount per ogni anno riferita alla somma totale dei loans
df_group["percentage_amount"] = (df_group["loan_amount"]/df_group["loan_amount"].sum())*100


df_group = df_group.reset_index()

time: 221 ms


In [19]:
df_group.head()

,index,count,loan_amount,percentage_amount
0,2005,203.000,102850.000,0.009
1,2006,2172.000,1376575.000,0.117
2,2007,24400.000,15446525.000,1.307
3,2008,54586.000,39423050.000,3.337
4,2009,83076.000,59689475.000,5.052


time: 7.71 ms


### 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [20]:
# Calcolo, per ogni loan, del numero dei suoi partecipanti
df_n_lenders = df_loan_lender_norm.groupby('loan_id').count().reset_index()
df_n_lenders = df_n_lenders.rename(columns= {'lenders': 'n_lenders'})
df_n_lenders.head()

,loan_id,n_lenders
0,84,3
1,85,2
2,86,3
3,88,3
4,89,4


time: 3.45 s


In [21]:
# Calcolo, per ogni loan, del numero dei suoi partecipanti
df_n_lenders = df_loan_lender_norm.groupby('loan_id').count().reset_index()
df_n_lenders = df_n_lenders.rename(columns= {'lenders': 'n_lenders'})

# merge dei dataframe
df_merge = pd.merge(df_loans[['loan_id', 'loan_amount']], df_loan_lender_norm, on="loan_id")
df_merge = pd.merge(df_merge, df_n_lenders, on="loan_id")

# Calcolo dell'ammontare di ogni prestatore per ogni prestito in modo equo
df_merge['amount_per_lender'] = df_merge['loan_amount'] / df_merge["n_lenders"]

# Raggruppo per lender e calcolo la somma dei prestiti dati
df_groupby = df_merge.groupby("lenders")["amount_per_lender"].sum().reset_index()

time: 59.4 s


In [22]:
df_groupby.head()

,lenders,amount_per_lender
0,000,1764.285
1,00000,1380.694
2,0002,2472.564
3,00mike00,52.632
4,0101craign0101,2623.565


time: 17.9 ms


### 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [23]:
# Nel dataset c'è un numero elevato di dati mancanti per la colonna country_code
df_lenders.info(null_counts=True)

print("\n--------------------------------------------------------------\n")

# Inoltre si vede che nel dataset dei loans la colonna country_code ha 9 valori mancanti, 
# mentre country_name non ne ha 
df_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349174 entries, 0 to 2349173
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   permanent_name  2349174 non-null  object
 1   city            729868 non-null   object
 2   state           635693 non-null   object
 3   country_code    890539 non-null   object
dtypes: object(4)
memory usage: 71.7+ MB

--------------------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419607 entries, 0 to 1419606
Data columns (total 8 columns):
 #   Column                   Non-Null Count    Dtype              
---  ------                   --------------    -----              
 0   loan_id                  1419607 non-null  int64              
 1   loan_name                1372049 non-null  object             
 2   loan_amount              1419607 non-null  float64            
 3   country_code             1419598 non-null  object

In [24]:
# I valori mancanti in df_loans sono associati tutti allo stato della Namibia
df_loans[df_loans['country_code'].isna()]

,loan_id,loan_name,loan_amount,country_code,country_name,planned_expiration_time,disburse_time,diff_days
82889,991853,Elizabeth,3325.000,NaN,Namibia,2016-01-21 02:20:06+00:00,2016-01-05 08:00:00+00:00,15.000
156970,513472,Gottlieb,7150.000,NaN,Namibia,2013-02-02 00:10:01+00:00,2013-03-08 08:00:00+00:00,-35.000
598087,851360,Elizabeth,4150.000,NaN,Namibia,2015-04-24 14:00:04+00:00,2015-06-03 07:00:00+00:00,-40.000
684876,1068159,Theo,4000.000,NaN,Namibia,2016-06-14 14:10:06+00:00,2016-06-10 07:00:00+00:00,4.000
971827,998555,Theo,3325.000,NaN,Namibia,2016-01-21 02:20:17+00:00,2016-01-05 08:00:00+00:00,15.000
1134818,1147866,Ndeyapo,5000.000,NaN,Namibia,2016-10-19 01:50:05+00:00,2016-10-01 07:00:00+00:00,17.000
1214923,851368,Theo,4150.000,NaN,Namibia,2015-04-24 14:00:04+00:00,2015-06-03 07:00:00+00:00,-40.000
1281022,1147852,Dominic,5100.000,NaN,Namibia,2016-11-02 18:00:07+00:00,2016-10-01 07:00:00+00:00,32.000
1415763,1068167,Elizabeth,3325.000,NaN,Namibia,2016-06-14 16:30:07+00:00,2016-06-10 07:00:00+00:00,4.000


time: 86.1 ms


In [25]:
# Analogo discorso per i record dei lenders, dove tutti i valori della Namibia sono mancanti
df_lenders[df_lenders['state']=='Namibia']

,permanent_name,city,state,country_code
2063842,jakeskaova9324,Windhoek,Namibia,NaN
2158590,doreen3422,Windhoek,Namibia,NaN
2273721,ckqdm1115,Windhoek,Namibia,NaN


time: 144 ms


In [26]:
# Per evitare di perdere tutte le informazioni legate ad un intero paese,
# solo in questo caso si procede con un rimpiazzo manuale dei missing value
df_lenders.loc[df_lenders['state']=='Namibia', 'country_code'] = 'Namibia'
df_loans.loc[df_loans['country_code'].isna(), 'country_code'] = 'Namibia'

time: 195 ms


In [27]:
# Si separano i lenders con valore di country_code presente dai lenders con valore country_code mancante
df_lenders_no_na = df_lenders[df_lenders['country_code'].notna()]

# Si ottiene una serie di booleani per poter eseguire la sostistuzione dei valori mancanti successivamente
sr_lenders_na = df_lenders['country_code'].isna()

time: 277 ms


In [28]:
# Si trova la distribuzione dei lender con valore di country_code presente 
# utilizzando l'opzione normalize di value_count che calcola la frequenza relativa dei valori
sr_state_distr = df_lenders_no_na['country_code'].value_counts(normalize=True)

# Si usa la distribuzione trovata per assegnare lo stato per i valori mancanti di country_code.
# Si usa la funzione choice utilizzando come valori i codici dei paesi, come dimensione del campione la somma dei
# booleani della serie che distingue tra codice na e non na e per probabilità la distribuzione trovata.
df_lenders.loc[sr_lenders_na, "country_code"] = np.random.choice(sr_state_distr.index, size=sum(sr_lenders_na), 
                                                                 p=sr_state_distr.values)

time: 457 ms


In [29]:
# si verifico che non ci siano valori ancanti
sum(df_lenders.country_code.isna())

0

time: 312 ms


In [30]:
# Si effettua il merge dei lenders con quanto hanno prestato in totale
df_lent = pd.merge(df_lenders, df_groupby, left_on='permanent_name', right_on='lenders')

time: 2.37 s


In [31]:
# Si sommano i prestiti raggruppati per paese
df_lent_country = df_lent.groupby('country_code')['amount_per_lender'].sum().reset_index(name='amount_lent')

time: 406 ms


In [32]:
df_lent_country.head()

,country_code,amount_lent
0,AD,6872.472
1,AE,6948972.075
2,AF,252855.879
3,AG,1146.075
4,AI,677.441


time: 8.93 ms


In [33]:
# Si calcola il totale dei prestiti ricevuti per ogni paese
df_borrowed = df_loans.groupby('country_code')['loan_amount'].sum().reset_index(name='borrowed')

time: 85.6 ms


In [34]:
df_borrowed.head()

,country_code,borrowed
0,AF,1967950.000
1,AL,4307350.000
2,AM,22950475.000
3,AZ,14784625.000
4,BA,477250.000


time: 6.72 ms


In [35]:
# Si effettua il merge dei due dataset
df_merge = pd.merge(df_lent_country, df_borrowed, on="country_code", how='outer')

# Si sostituiscono i valori mancanti con 0
df_merge.fillna(0, inplace=True)

# Si calcola la differenza tra il denaro prestato e quello preso in prestito
df_merge['diff_lent-borrowed'] = df_merge['amount_lent'] - df_merge['borrowed']

time: 504 ms


In [36]:
df_merge.head()

,country_code,amount_lent,borrowed,diff_lent-borrowed
0,AD,6872.472,0.000,6872.472
1,AE,6948972.075,0.000,6948972.075
2,AF,252855.879,1967950.000,-1715094.121
3,AG,1146.075,0.000,1146.075
4,AI,677.441,0.000,677.441


time: 9.79 ms


### 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [37]:
# Come nel punto precedente, il codice dello stato della Namibia è assente
df_country_stats[df_country_stats['country_name'] == 'Namibia']

,country_name,country_code,population,population_below_poverty_line
115,Namibia,NaN,2533794,28.700


time: 13.7 ms


In [38]:
# Per non perdere le sue informazioni lo si inserisce manualmente
df_country_stats.loc[df_country_stats['country_name'] == 'Namibia', 'country_code'] = 'Namibia'

time: 4.14 ms


In [39]:
# Si effettua il merge del dataset delle informazioni degli stati con quello della differenza di prestiti 
# calcolato al punto precedente
df_money_state = pd.merge(df_merge, df_country_stats, on="country_code")

# Calcolo del rapporto richiesto
df_money_state['ratio_diff_L-B_pop'] = df_money_state['diff_lent-borrowed']/df_money_state['population']

time: 9.58 ms


In [40]:
df_money_state.head()

,country_code,amount_lent,borrowed,diff_lent-borrowed,country_name,population,population_below_poverty_line,ratio_diff_L-B_pop
0,AE,6948972.075,0.000,6948972.075,United Arab Emirates,9400145,19.500,0.739
1,AF,252855.879,1967950.000,-1715094.121,Afghanistan,35530081,35.800,-0.048
2,AL,33641.267,4307350.000,-4273708.733,Albania,2930187,14.300,-1.459
3,AM,38616.075,22950475.000,-22911858.925,Armenia,2930450,32.000,-7.819
4,AO,38386.633,0.000,38386.633,Angola,29784193,40.500,0.001


time: 18.1 ms


In [41]:
# Si trova lo stato con il rapport più grande
df_money_state.iloc[[df_money_state['ratio_diff_L-B_pop'].idxmax()]]

,country_code,amount_lent,borrowed,diff_lent-borrowed,country_name,population,population_below_poverty_line,ratio_diff_L-B_pop
114,NO,21475967.777,0.000,21475967.777,Norway,5305383,nan,4.048


time: 18.4 ms


### 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [42]:
# La colonna population_below_poverty_line è espressa in percentuale,
# Si calcola la percentuale di persone sopra la soglia di poverà
df_money_state['population_above_poverty_line'] = 100 - df_money_state['population_below_poverty_line']

# Si calcola la quantità di popolazione al di sotto della fascia di povertà utilizzando la popolazione totale 
# e la percentuale calcolata sopra
pop_above_poverty_line = (df_money_state['population']*df_money_state['population_above_poverty_line'])/100

# Si calcola il rapport richiesto
df_money_state['ratio_diff_L-B_pop_above_poverty'] = df_money_state['diff_lent-borrowed']/pop_above_poverty_line

time: 6.88 ms


In [43]:
df_money_state.head()

,country_code,amount_lent,borrowed,diff_lent-borrowed,country_name,population,population_below_poverty_line,ratio_diff_L-B_pop,population_above_poverty_line,ratio_diff_L-B_pop_above_poverty
0,AE,6948972.075,0.000,6948972.075,United Arab Emirates,9400145,19.500,0.739,80.500,0.918
1,AF,252855.879,1967950.000,-1715094.121,Afghanistan,35530081,35.800,-0.048,64.200,-0.075
2,AL,33641.267,4307350.000,-4273708.733,Albania,2930187,14.300,-1.459,85.700,-1.702
3,AM,38616.075,22950475.000,-22911858.925,Armenia,2930450,32.000,-7.819,68.000,-11.498
4,AO,38386.633,0.000,38386.633,Angola,29784193,40.500,0.001,59.500,0.002


time: 18.2 ms


In [44]:
# Si trova lo stato con il rapport più grande
df_money_state.iloc[[df_money_state['ratio_diff_L-B_pop_above_poverty'].idxmax()]]

,country_code,amount_lent,borrowed,diff_lent-borrowed,country_name,population,population_below_poverty_line,ratio_diff_L-B_pop,population_above_poverty_line,ratio_diff_L-B_pop_above_poverty
24,CA,103849471.667,50000.000,103799471.667,Canada,36624199,9.400,2.834,90.600,3.128


time: 17 ms


### 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

Ci sono tre possibili casi da considerare:
1. i loans per cui le date di disburse_time e planned_expiration_time sono presenti e sono coerenti, 
   cioè quando planned_expiration_time > disburse_time
2. i loans in cui le due date sono presenti ma si ha planned_expiration_time > disburse_time
3. i loans per cui almeno una delle due date non è presente

Il primo passaggio consiste ne separare le righe del caso 3 dalle altre, per questi loans non è possibile 
calcolare i dati richiesti, si sostiuiranno i dati mancanti con 0.

Il passo successivo sarà distinguere i loans appartenenti al caso 1 da quelli appartenenti al caso 2,
nel primo caso si procederà per il calcolo dell'ammontere per anno.

Per i lenders che appartengono al caso 2, dato che non è possibile ricevere un prestito dopo la data della 
sua scadenza, si considerano i due valori come identici, in particolare uguali a disburse_time.

In [45]:
# Si separano i lender con valori mancanti dagli altri
df_loans_na = df_loans[df_loans["disburse_time"].isna() | df_loans['planned_expiration_time'].isna()]
df_loans_no_na = df_loans.dropna(subset=["disburse_time", "planned_expiration_time"])

time: 286 ms


In [46]:
# info dei due dataframe
df_loans_na.info()
print()
print("-------------------------------------------------------------")
print()
df_loans_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374645 entries, 64 to 1419564
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype              
---  ------                   --------------   -----              
 0   loan_id                  374645 non-null  int64              
 1   loan_name                358121 non-null  object             
 2   loan_amount              374645 non-null  float64            
 3   country_code             374645 non-null  object             
 4   country_name             374645 non-null  object             
 5   planned_expiration_time  2811 non-null    datetime64[ns, UTC]
 6   disburse_time            371832 non-null  datetime64[ns, UTC]
 7   diff_days                0 non-null       float64            
dtypes: datetime64[ns, UTC](2), float64(2), int64(1), object(3)
memory usage: 25.7+ MB

-------------------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1044962 entries, 

In [47]:
# Si gestiscono i casi in cui le due date non sono corenti
loans_inc = df_loans_no_na['disburse_time'] > df_loans_no_na['planned_expiration_time']

df_loans_no_na.loc[loans_inc, 'planned_expiration_time'] = df_loans_no_na.loc[loans_inc, 'disburse_time']

time: 31 ms


/home/simone/.local/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [48]:
def get_amount_x_year(row):
    '''
    Riceve in input una riga del dataset e calcola, per ogni anno tra disburse_time e planned_expiration_time,
    la quantità di giorni di quell'anno. Nel caso di disburse_time considera i giorni dalla data fino alla fine 
    dell'anno, nel caso di planned_expiration_time considera i giorni passati dall'inizio dell'anno alla data in 
    questione. Ritorna un dizionario le cui chiavi sono gli anni all'interno del periodo considerato e i valori
    sono l'ammontare del loan per quell'anno dell'anno calcolati come descritto.
    
    ...

    Parameters
    ----------
    row : pandas.core.series.Series
        La riga del dataset
    
    
    Returns
    -------
    dict
        dizionario di con le informazioni del loan(id, name, amount), i giorni del prestito e per ogni anno
        l'importo in base ai giorni trascorsi
    '''
    
    # Si estraggono gli anni in mezzo
    planned = row["planned_expiration_time"]
    disburse = row["disburse_time"]
    
    # Calcolo degli anni tra disburse e planned esclusi loro stessi
    range_year = list(range(disburse.year + 1, planned.year))
    
    tot_amount = row['loan_amount']
    tot_days = (planned - disburse).days + 1
    
    amount_x_year = dict()
    
    # Si inseriscono i dati iniziali del prestito
    amount_x_year.update({
        'loan_id': row['loan_id'], 
        'loan_name': row['loan_name'], 
        'loan_amount': tot_amount,
        'days_loan': tot_days
    })
    
    # Se planned e disburse hanno lo stesso anno, considero solo la loro differenza in giorni
    if planned.year == disburse.year:
        amount_x_year[planned.year] = tot_amount
        
    # Altrimenti se gli anni sono differenti si aggiungono entrambi
    else:
        # Per disburse devo calcolare quelli che rimangono fino alla fine dell'anno,
        # Si calcolano i giorni totali dell'anno in questione che vengono tolti ai giorni dell'anno passati 
        # fino a disburse
        days_x_year = pd.Period("{}-12-31".format(disburse.year)).dayofyear - disburse.dayofyear
        
        # Calcolo dell'ammontare per l'anno disburse
        amount_x_year[disburse.year] = tot_amount*(days_x_year/tot_days)
        
        # Per planned si calcolano i giorni passati fino alla data in questione
        # e l'ammontare per l'anno planned
        amount_x_year[planned.year] = tot_amount*((planned.dayofyear)/tot_days)
        
        # Per gli anni all'interno del range aggiungo coppia (anno, ammontere per quell'anno) al dict
        if range_year:
            amount_x_year.update({
                k: tot_amount*(pd.Period("{}-12-31".format(k)).dayofyear/tot_days) for k in range_year
            })
            
    return amount_x_year


time: 3.55 ms


In [49]:
# Calcolo del prestito per ogni anno
res = df_loans_no_na.apply(lambda x: get_amount_x_year(x), axis=1)

time: 5min 17s


In [50]:
# Creazione del dataframe e riordino delle colonne
df_res = pd.DataFrame(list(res))
df_res.columns = list(map(str, df_res.columns))
cols_reorder = ['loan_id', 'loan_name', 'loan_amount', 'days_loan', '2011', 
                '2012', '2013', '2014', '2015', '2016', '2017', '2018']
df_res = df_res.reindex(cols_reorder, axis=1)


time: 2.54 s


In [51]:
df_res.head()

,loan_id,loan_name,loan_amount,days_loan,2011,2012,2013,2014,2015,2016,2017,2018
0,657307,Aivy,125.000,54,nan,nan,20.833,104.167,nan,nan,nan,nan
1,657259,Idalia Marizza,400.000,97,nan,nan,45.361,350.515,nan,nan,nan,nan
2,658010,Aasia,400.000,38,nan,nan,nan,400.000,nan,nan,nan,nan
3,659347,Gulmira,625.000,35,nan,nan,nan,625.000,nan,nan,nan,nan
4,656933,Ricky\t,425.000,58,nan,nan,102.586,322.414,nan,nan,nan,nan


time: 14.6 ms


In [52]:
# Gestione dei record con almeno una delle due date mancanti, si creano le colonne degli anni e dei giorni passati
# che contengono solo 0
df_na_sel = df_loans_na[['loan_id', 'loan_name','loan_amount']]
df_na_sel[['days_loan', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']] = pd.DataFrame(
    [np.full([9], np.nan)], index=df_na_sel.index)

time: 52.7 ms


/home/simone/.local/lib/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [53]:
# Concatenazione dei due dataset
df_res = pd.concat([df_res, df_na_sel])

time: 119 ms


In [54]:
df_res.head()

,loan_id,loan_name,loan_amount,days_loan,2011,2012,2013,2014,2015,2016,2017,2018
0,657307,Aivy,125.000,54.000,nan,nan,20.833,104.167,nan,nan,nan,nan
1,657259,Idalia Marizza,400.000,97.000,nan,nan,45.361,350.515,nan,nan,nan,nan
2,658010,Aasia,400.000,38.000,nan,nan,nan,400.000,nan,nan,nan,nan
3,659347,Gulmira,625.000,35.000,nan,nan,nan,625.000,nan,nan,nan,nan
4,656933,Ricky\t,425.000,58.000,nan,nan,102.586,322.414,nan,nan,nan,nan


time: 15.2 ms


In [55]:
df_res[['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']].sum().reset_index(name ='Total Amount')

,index,Total Amount
0,2011,622359.672
1,2012,113142070.848
2,2013,124721556.642
3,2014,152878203.817
4,2015,159558536.468
5,2016,158978167.401
6,2017,172278351.286
7,2018,5895951.398


time: 190 ms


In [56]:
final = time.time() - start

time: 677 µs


In [57]:
final

455.70880484580994

time: 3 ms


In [58]:
456/60

7.6

time: 2.55 ms
